In [1]:
# load the document
from langchain_community.document_loaders import  PDFPlumberLoader
import re


In [9]:
import pdfplumber

pages = []
with pdfplumber.open(r"../document/GCC_e78f9d59-77bb-4de7-99f41730977875550_Shefali_Singh.pdf") as pdf:
    for page in pdf.pages:
        # Extract text
        text = page.extract_text()
        
        # Extract tables
        tables = page.extract_tables()
        
        page_info = {
            "text" : text,
            "tables" : tables,
            "page_number" : page.page_number
        }
        
        pages.append(page_info)

In [10]:
for page in pages[1:]:
    # Extract the text of every page
    text = page['text']
    # remove the Indian Oil Corporation General Conditions of Contract\n from every text
    header = 'Indian Oil Corporation General Conditions of Contract\n'
    text = re.sub(r'^' + re.escape(header), '', text)
    # Remove footer page number if present
    text = re.sub(r'\n\d+\s*$', '', text)
    # Replace the page text with this one
    page['text'] = text

In [11]:
# Now combine the texts into one single list to check them afterwards
all_page_texts = []
for page in pages:
    if page['text']:
        all_page_texts.append(page['text'])

full_texts = "\n".join(all_page_texts)

In [12]:
# Use recursive text splitter to divide the text into chuncks
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Initialise the text splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# Storage for all chunks
chunks = splitter.create_documents([full_texts])

In [13]:
chunks

[Document(metadata={}, page_content='INDIAN OIL CORPORATION LIMITED\nGENERAL CONDITIONS OF CONTRACT\n(AS AMENDED IN JANUARY 2003)\nGENERAL CONDITIONS OF CONTRACT\nSECTION 1\nDEFINITIONS\n1.0.0.0 The following expressions hereunder and elsewhere in the Contract Documents used, unless\nrepugnant to the subject or context thereof, shall have the following meanings hereunder\nrespectively assigned to them, namely:\n1.1.0.0 "Acceptance of Tender" shall mean the Acceptance of Tender issued by the OWNER to the\nCONTRACTOR, and shall include a letter, telegram or fax of acceptance or other notification of\naward of work, and a detailed Letter of Acceptance.\n1.2.0.0 "Approval" shall mean the written and signed approval of the OWNER or of Engineer-in-Charge or\nConsultant authorised in this behalf by the OWNER, and with respect to a plan or drawing shall\ninclude an approval in Code 2, subject to the limitation(s) specified in such approval.\n1.3.0.0 "Approval in Code 2" shall mean an approval 

In [14]:
# Create and import langchain ollamaembeddings anbd chroma db vector stores
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

In [15]:
embed = OllamaEmbeddings(model="nomic-embed-text")

In [16]:
# Create the chromadb vector store
vectorstore = Chroma.from_documents(
    documents=chunks,
    embeddings=embed,
    persist_directory='./chroma_db'
)

TypeError: Chroma.__init__() got an unexpected keyword argument 'embeddings'